# Implicit & LightFM

В данном jupyter notebook рассматриваются как использовать implicit и LightFM для построения рекомендаций

* [Load data](#load-data)
* [Validation](#validation)
* [Implicit](#implicit)
* [LightFM](#lightfm)
* [Links](#links)

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.sparse as sp
from itertools import islice, cycle
from more_itertools import pairwise
from tqdm.auto import tqdm


<a id="load-data"></a>
# Load data

Возьмем уже предобработанный данные из [metrics-validation-strategies-and-baselines](https://www.kaggle.com/sharthz23/metrics-validation-strategies-and-baselines/output)

In [ ]:
df = pd.read_pickle('interactions_preprocessed.pickle')
df_users = pd.read_pickle('users_preprocessed.pickle')
df_items = pd.read_pickle('items_preprocessed.pickle')

df.shape, df_users.shape, df_items.shape

((1532998, 5), (142888, 3), (59599, 5))

In [ ]:
display(df.head())
display(df_users.head())
display(df_items.head())

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [ ]:
#меппинг айди юзера и будущей строки? порядковый номер
users_inv_mapping = dict(enumerate(df['user_id'].unique()))
#reverse
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

151600

In [ ]:
users_mapping

{126706: 0,
 127290: 1,
 66991: 2,
 46791: 3,
 79313: 4,
 63454: 5,
 127451: 6,
 42797: 7,
 47287: 8,
 23439: 9,
 47551: 10,
 42856: 11,
 59484: 12,
 142154: 13,
 64211: 14,
 7074: 15,
 14569: 16,
 65636: 17,
 125501: 18,
 115029: 19,
 129929: 20,
 80469: 21,
 88088: 22,
 18548: 23,
 91177: 24,
 1011: 25,
 8569: 26,
 70824: 27,
 33450: 28,
 137220: 29,
 11142: 30,
 125178: 31,
 50172: 32,
 49508: 33,
 64786: 34,
 133008: 35,
 85714: 36,
 105213: 37,
 39623: 38,
 2953: 39,
 32533: 40,
 144602: 41,
 77247: 42,
 132581: 43,
 2129: 44,
 67255: 45,
 73724: 46,
 88118: 47,
 3857: 48,
 89204: 49,
 152601: 50,
 13255: 51,
 63959: 52,
 89035: 53,
 51355: 54,
 157504: 55,
 12230: 56,
 40350: 57,
 141998: 58,
 52841: 59,
 109896: 60,
 127918: 61,
 130323: 62,
 54486: 63,
 7855: 64,
 135292: 65,
 93981: 66,
 57170: 67,
 137065: 68,
 82644: 69,
 63549: 70,
 99430: 71,
 48543: 72,
 22827: 73,
 142977: 74,
 98495: 75,
 36257: 76,
 154822: 77,
 68949: 78,
 91527: 79,
 59362: 80,
 50171: 81,
 14500: 82

In [ ]:
# такая же история для мэппинга айтемов
items_inv_mapping = dict(enumerate(df['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

59599

In [ ]:
df_items['title'] = df_items['title'].str.lower()

In [ ]:
item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()
#матчим имя фильма и айди фильма
len(item_titles), item_titles[128115]

(59599, 'ворон-челобитчик')

In [ ]:
#есть тайтлы с большим количеством айдишников (фильмы, которые одинаково называются)
title_items = df_items.groupby('title')['id'].agg(list)
title_items

title
# 20 восьмая                                                     [201623]
# duo                                                             [72582]
# me too. роман                                                  [171172]
# партия                                                         [224512]
#1917: человек из раньшего времени. библиотека «проекта 1917»    [230768]
                                                                   ...   
…чума на оба ваши дома!                                          [226481]
№ 12, или история одного прекрасного юноши                        [20979]
伦巴德人的故事                                                          [119226]
地球への旅                                                            [148400]
�baby blues�                                                      [98635]
Name: id, Length: 57289, dtype: object

In [ ]:
title_count = title_items.map(len)
title_count.value_counts()

1     55708
2      1197
3       245
4        71
5        38
6        11
7         8
8         3
9         2
18        1
11        1
13        1
23        1
12        1
47        1
Name: id, dtype: int64

In [ ]:
title_items[title_count > 1].tail()

title
яма                                              [60156, 165785]
янки из коннектикута при дворе короля артура      [14759, 56530]
японская диета                                   [168986, 74652]
яр                                                [168761, 5371]
ящик пандоры                                    [236465, 158851]
Name: id, dtype: object

In [ ]:
df_items[df_items['title'] == 'ящик пандоры']

,id,title,genres,authors,year
40426,236465,ящик пандоры,"Любовно-фантастические романы,Научная фантастика",Филипп Хорват,2017
54854,158851,ящик пандоры,"Мистика,Современная зарубежная литература",Бернар Вербер,2018


In [ ]:
title_items[title_count > 1].head()

title
(о переводе)                 [61213, 145263]
2084                        [177082, 281410]
451 градус по фаренгейту     [44681, 162716]
playboy 02-2018               [114405, 8599]
playboy 03-2018              [31097, 121747]
Name: id, dtype: object

In [ ]:
df_items[df_items['title'] == '451 градус по фаренгейту']

,id,title,genres,authors,year
19398,44681,451 градус по фаренгейту,"Социальная фантастика,Зарубежная фантастика,На...",Рэй Брэдбери,1953
50150,162716,451 градус по фаренгейту,"Социальная фантастика,Зарубежная фантастика,На...",Рэй Брэдбери,"1951, 1953, 1967"


In [ ]:
#для оптимизации памяти
df['rating'] = np.array(df['rating'].values, dtype=np.float32)

df.loc[df['item_id'].isin([44681, 162716])].groupby('item_id').agg({
    'progress': np.size,
    'rating': ['mean'],
    'start_date': ['min', 'max'],
})

progress rating start_date           
            size   mean        min        max
item_id                                      
44681        353   4.56 2018-01-24 2019-12-20
162716        59   4.80 2018-01-25 2019-12-30

<a id="validation"></a>
# Validation

Для наших данных выбрем 7 последних дней и будем тестировать на них последовательно (1 test fold - 1 день).

Но теперь нам нужно учитывать проблему холодного старта. Это основная проблем классических метод над матрицей взаимодействий.
Поэтому напишем свой класс для разбиения исходного датафрейма на train/test

In [ ]:
class TimeRangeSplit():
    """
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
    """
    def __init__(self,
                 start_date,
                 end_date=None,
                 freq='D',
                 periods=None,
                 tz=None,
                 normalize=False,
                 closed=None,
                 train_min_date=None,
                 filter_cold_users=True,
                 filter_cold_items=True,
                 filter_already_seen=True):

        self.start_date = start_date
        if end_date is None and periods is None:
            raise ValueError("Either 'end_date' or 'periods' must be non-zero, not both at the same time.")

        self.end_date = end_date
        self.freq = freq
        self.periods = periods
        self.tz = tz
        self.normalize = normalize
        self.closed = closed
        self.train_min_date = pd.to_datetime(train_min_date, errors='raise')
        self.filter_cold_users = filter_cold_users
        self.filter_cold_items = filter_cold_items
        self.filter_already_seen = filter_already_seen

        self.date_range = pd.date_range(
            start=start_date,
            end=end_date,
            freq=freq,
            periods=periods,
            tz=tz,
            normalize=normalize,
            closed=closed)

        self.max_n_splits = max(0, len(self.date_range) - 1)
        if self.max_n_splits == 0:
            raise ValueError("Provided parametrs set an empty date range.")

    def split(self,
              df,
              user_column='user_id',
              item_column='item_id',
              datetime_column='date',
              fold_stats=False):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            train_min_mask = df_datetime >= self.train_min_date
        else:
            train_min_mask = df_datetime.notnull()

        date_range = self.date_range[(self.date_range >= df_datetime.min()) &
                                     (self.date_range <= df_datetime.max())]

        for start, end in pairwise(date_range):
            fold_info = {
                'Start date': start,
                'End date': end
            }
            train_mask = train_min_mask & (df_datetime < start)
            train_idx = df.index[train_mask]
            if fold_stats:
                fold_info['Train'] = len(train_idx)

            test_mask = (df_datetime >= start) & (df_datetime < end)
            test_idx = df.index[test_mask]

            if self.filter_cold_users:
                new = np.setdiff1d(
                    df.loc[test_idx, user_column].unique(),
                    df.loc[train_idx, user_column].unique())
                new_idx = df.index[test_mask & df[user_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New users'] = len(new)
                    fold_info['New users interactions'] = len(new_idx)

            if self.filter_cold_items:
                new = np.setdiff1d(
                    df.loc[test_idx, item_column].unique(),
                    df.loc[train_idx, item_column].unique())
                new_idx = df.index[test_mask & df[item_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New items'] = len(new)
                    fold_info['New items interactions'] = len(new_idx)

            if self.filter_already_seen:
                user_item = [user_column, item_column]
                train_pairs = df.loc[train_idx, user_item].set_index(user_item).index
                test_pairs = df.loc[test_idx, user_item].set_index(user_item).index
                intersection = train_pairs.intersection(test_pairs)
                test_idx = test_idx[~test_pairs.isin(intersection)]
                # test_mask = rd.df.index.isin(test_idx)
                if fold_stats:
                    fold_info['Known interactions'] = len(intersection)

            if fold_stats:
                fold_info['Test'] = len(test_idx)

            yield (train_idx, test_idx, fold_info)

    def get_n_splits(self, df, datetime_column='date'):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            df_datetime = df_datetime[df_datetime >= self.train_min_date]

        date_range = self.date_range[(self.date_range >= df_datetime.min()) &
                                     (self.date_range <= df_datetime.max())]

        return max(0, len(date_range) - 1)

In [ ]:
last_date = df['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [ ]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']

    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

In [ ]:
cv = TimeRangeSplit(start_date=start_date, periods=folds+1)

cv.max_n_splits, cv.get_n_splits(df, datetime_column='start_date')

<ipython-input-18-6d72e7cc5cb7>:33: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  self.date_range = pd.date_range(


(7, 7)

In [ ]:
cv.date_range

DatetimeIndex(['2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
               '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
folds_with_stats = list(cv.split(
    df,
    user_column='user_id',
    item_column='item_id',
    datetime_column='start_date',
    fold_stats=True
))

folds_info_with_stats = pd.DataFrame([info for _, _, info in folds_with_stats])

In [ ]:
folds_info_with_stats

,Start date,End date,Train,New users,New users interactions,New items,New items interactions,Known interactions,Test
0,2019-12-24,2019-12-25,1515946,3,3,0,0,0,2045
1,2019-12-25,2019-12-26,1517994,1,1,0,0,0,2141
2,2019-12-26,2019-12-27,1520136,0,0,0,0,0,2177
3,2019-12-27,2019-12-28,1522313,0,0,0,0,0,2110
4,2019-12-28,2019-12-29,1524423,2,4,0,0,0,2205
5,2019-12-29,2019-12-30,1526632,4,4,0,0,0,2118
6,2019-12-30,2019-12-31,1528754,1,1,0,0,0,2168


In [ ]:
fold_dates = [(info['Start date'], info['End date']) for _, _, info in folds_with_stats]
fold_dates

[(Timestamp('2019-12-24 00:00:00', freq='D'),
  Timestamp('2019-12-25 00:00:00', freq='D')),
 (Timestamp('2019-12-25 00:00:00', freq='D'),
  Timestamp('2019-12-26 00:00:00', freq='D')),
 (Timestamp('2019-12-26 00:00:00', freq='D'),
  Timestamp('2019-12-27 00:00:00', freq='D')),
 (Timestamp('2019-12-27 00:00:00', freq='D'),
  Timestamp('2019-12-28 00:00:00', freq='D')),
 (Timestamp('2019-12-28 00:00:00', freq='D'),
  Timestamp('2019-12-29 00:00:00', freq='D')),
 (Timestamp('2019-12-29 00:00:00', freq='D'),
  Timestamp('2019-12-30 00:00:00', freq='D')),
 (Timestamp('2019-12-30 00:00:00', freq='D'),
  Timestamp('2019-12-31 00:00:00', freq='D'))]

<a id="lightfm"></a>
# LightFM

Библиотека для построения рекомендательных систем с помощью матричных разложений и фичей
* Репо - https://github.com/lyst/lightfm
* Документация - https://making.lyst.com/lightfm/docs/home.html

Входные данные - разреженные матрицы (либо csr_matrix, либо coo_matrix)
Модель одна, варируются только loss functions. Также есть вспомогательный класс lightfm.data.Dataset для построения как матрицы взаимодействий, так и матриц фичей



```
# Выбран кодовый формат
```

**Основая идея** - построить векторные представления для каждой фичи по пользователю и объекту. Если фичей нет, то просто используем индикаторные фичи (единичная матрица по пользователям и объектам соответственно)
![image.png](attachment:image.png)
[image source](https://sites.northwestern.edu/msia/2019/04/24/personalized-restaurant-recommender-system-using-hybrid-approach/#:~:text=LightFM%20incorporates%20matrix%20factorization%20model,retain%20the%20original%20interaction%20matrix.&text=The%20latent%20embeddings%20could%20capture,items%2C%20which%20represent%20their%20tastes.)

**Векорное представление** - векторы для пользователя и объекта получают суммирование векторов их фичей.

**Процесс построения рекомендаций** - для конкретного пользователя ищутся такие объекты, что скалярное произведение их векторов максимально.

**Пространство для тюнинга**
* взвешивание на уровне таблицы
* `loss` - обучение функции ранжирования.
    * `logistic`
    * `bpr`
    * `warp`
    * `warp-kos`
* `no_components` - размерность итоговых векторов (обычно степени 2-ки, от 16 до 256)
* `learning_rate` - "скорость" обучения
* `item_alpha`/`user_alpha` - регуляризация (степени 10-ки, от 0.0001 до 1)

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=7b0eab14e972bcf2ee195c12e48e0d56f8644a5c2cf7c6ba6c09e8da0a80eb94
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
from lightfm.data import Dataset
from lightfm import LightFM

In [ ]:
dataset = Dataset()

In [ ]:
dataset.fit(df['user_id'].unique(), df['item_id'].unique())

In [ ]:
df_users['age'] = df_users['age'].cat.add_categories('age_unknown')
df_users['age'] = df_users['age'].fillna('age_unknown')
age_features = df_users['age'].unique()
age_features

['45_54', '18_24', '65_inf', '35_44', '55_64', '25_34', 'age_unknown']
Categories (7, object): ['18_24', '25_34', '35_44', '45_54', '55_64', '65_inf', 'age_unknown']

In [ ]:
df_users['sex'] = np.array(df_users['sex'].astype(str))
df_users['sex'] = df_users['sex'].fillna('age_unknown')
sex_features = df_users['sex'].unique()
sex_features

<ipython-input-31-e0dad7683477>:1: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  df_users['sex'] = np.array(df_users['sex'].astype(str))


array(['nan', '0.0', '1.0'], dtype=object)

In [ ]:
users_features = np.append(age_features, sex_features)
users_features

array(['45_54', '18_24', '65_inf', '35_44', '55_64', '25_34',
       'age_unknown', 'nan', '0.0', '1.0'], dtype=object)

In [ ]:
dataset.fit_partial(user_features=users_features)

In [ ]:
df_items['genres'] = df_items['genres'].cat.add_categories('genre_unknown')
df_items['genres'] = df_items['genres'].fillna('genre_unknown')
genres = list(df_items['genres'].str.split(',').explode().unique())
len(genres)

641

In [ ]:
dataset.fit_partial(item_features=genres)

In [ ]:
num_users, num_items = dataset.interactions_shape()
num_users, num_items

(151600, 59599)

In [ ]:
#активирует меппер
lightfm_mapping = dataset.mapping()


In [ ]:
lightfm_mapping = {
    #индекс к айди
    'users_mapping': lightfm_mapping[0],

    'user_features_mapping': lightfm_mapping[1],

    'items_mapping': lightfm_mapping[2],

    'item_features_mapping': lightfm_mapping[3],

}


print('users_mapping len - ', len(lightfm_mapping['users_mapping']))
print('user_features_mapping len - ', len(lightfm_mapping['user_features_mapping']))
print('items_mapping len - ', len(lightfm_mapping['items_mapping']))
print('Users item_features_mapping len - ', len(lightfm_mapping['item_features_mapping']))

users_mapping len -  151600
user_features_mapping len -  151610
items_mapping len -  59599
Users item_features_mapping len -  60240


In [ ]:
for key in lightfm_mapping.keys():
  print(key)
  print(len(lightfm_mapping[key]))


users_mapping
151600
user_features_mapping
151610
items_mapping
59599
item_features_mapping
60240


In [ ]:
lightfm_mapping['users_inv_mapping'] = {v: k for k, v in lightfm_mapping['users_mapping'].items()}
lightfm_mapping['items_inv_mapping'] = {v: k for k, v in lightfm_mapping['items_mapping'].items()}

In [ ]:
num_user_features = dataset.user_features_shape()
num_show_features = dataset.item_features_shape()
print('Num user features: {} -> {}\nnum item features: {} -> {}.'.format(
    num_user_features[1] - num_users, num_user_features[1],
    num_show_features[1] - num_items, num_show_features[1]))

Num user features: 10 -> 151610
num item features: 641 -> 60240.


Как собрать теперь наши матрицы.
* Для интеракций нужен итератор на
    * `[(user_id1, item_id1), (user_id2, item_id2), ...]`  
    * `[(user_id1, item_id1, weight), (user_id2, item_id2, weight), ...]`
* Для фичей нужен итератор на
    * `[(id, [feature1, feature2, ...]), (id, [feature1, feature2, ...]), ...]`  
    * `[(id, {feature1: weight}), (id, {feature1: weight}), ...]`

In [ ]:
def df_to_tuple_iterator(df):
    return zip(*df.values.T)

def concat_last_to_list(t):
    return (t[0], list(t[1:])[0])

def df_to_tuple_list_iterator(df):
    return map(concat_last_to_list, zip(*df.values.T))

In [ ]:
# Создание DataFrame
data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
data = pd.DataFrame(data)

In [ ]:
data

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [ ]:
train_idx, test_idx, info = folds_with_stats[0]

train = df.loc[train_idx]
test = df.loc[test_idx]

In [ ]:
#делаем матрицу с пересечениями из кучи кортежей
train_mat, train_mat_weights = dataset.build_interactions(df_to_tuple_iterator(train[['user_id', 'item_id']]))
train_mat

<151600x59599 sparse matrix of type '<class 'numpy.int32'>'
	with 1515946 stored elements in COOrdinate format>

In [ ]:
train_mat_weights

<151600x59599 sparse matrix of type '<class 'numpy.float32'>'
	with 1515946 stored elements in COOrdinate format>

In [ ]:
df_users['features'] = df_users[['age', 'sex']].astype(str).apply(lambda x: list(x), axis=1)
df_users.head()

,user_id,age,sex,features
0,1,45_54,nan,"[45_54, nan]"
1,2,18_24,0.0,"[18_24, 0.0]"
2,3,65_inf,0.0,"[65_inf, 0.0]"
3,4,18_24,0.0,"[18_24, 0.0]"
4,5,35_44,0.0,"[35_44, 0.0]"


In [ ]:
known_users_filter = df_users['user_id'].isin(df['user_id'].unique())
train_user_features = dataset.build_user_features(
    df_to_tuple_list_iterator(
        df_users.loc[known_users_filter, ['user_id', 'features']]
    )
)
train_user_features

<151600x151610 sparse matrix of type '<class 'numpy.float32'>'
	with 422954 stored elements in Compressed Sparse Row format>

In [ ]:
df_items['features'] = df_items['genres'].str.split(',')
df_items.head()

,id,title,genres,authors,year,features
0,128115,ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886,"[Зарубежные детские книги, Сказки, Зарубежная ..."
1,210979,скрипка ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894,"[Классическая проза, Литература 19 века, Русск..."
2,95632,испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869,"[Зарубежная классика, Классическая проза, Лите..."
3,247906,странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831,"[Пьесы и драматургия, Литература 19 века]"
4,294280,господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873,"[Зарубежная классика, Классическая проза, Лите..."


In [ ]:
df_items['features'].map(len).value_counts(dropna=False)

2     20734
1     20172
3     12699
4      4248
5      1074
6       373
7       146
8        74
9        34
10       22
11       12
13        6
12        2
17        2
16        1
Name: features, dtype: int64

In [ ]:
known_items_filter = df_items['id'].isin(df['item_id'].unique())

train_items_features = dataset.build_item_features(
    df_to_tuple_list_iterator(
        df_items.loc[known_items_filter, ['id', 'features']]
    )
)

train_items_features

<59599x60240 sparse matrix of type '<class 'numpy.float32'>'
	with 186360 stored elements in Compressed Sparse Row format>

In [ ]:
lfm_model = LightFM(no_components=64,
                    learning_rate=0.05, loss='warp',
                    max_sampled=5, random_state=23)

In [ ]:
num_epochs = 15
for _ in tqdm(range(num_epochs), total=num_epochs):
    lfm_model.fit_partial(
        train_mat,
        user_features=train_user_features,
        item_features=train_items_features,
        num_threads=4
    )

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
test#['user_id'].iloc[0]

,user_id,item_id,progress,rating,start_date
1515866,139942,26270,58,NaN,2019-12-24
1515867,136607,82392,39,NaN,2019-12-24
1515868,142287,84446,79,NaN,2019-12-24
1515869,114933,319155,9,NaN,2019-12-24
1515870,32926,120339,73,NaN,2019-12-24
...,...,...,...,...,...
1517909,138587,291806,0,NaN,2019-12-24
1517910,158991,99669,63,5.0,2019-12-24
1517911,77232,142149,4,NaN,2019-12-24
1517912,17843,174535,12,NaN,2019-12-24


In [ ]:
top_N = 10
#берем настоящий юзер айди
user_id = test['user_id'].iloc[0]

#берем номер строки из меппинга для этого пользователя
row_id = lightfm_mapping['users_mapping'][user_id]

print(f'Рекомендации для пользователя {user_id}, номер строки - {row_id}')

Рекомендации для пользователя 139942, номер строки - 10346


In [ ]:
#строим рекомендации для всех айтемов
all_cols = list(lightfm_mapping['items_mapping'].values())
len(all_cols)

59599

In [ ]:
pred = lfm_model.predict(row_id,
                         all_cols,
                         user_features=train_user_features,
                         item_features=train_items_features,
                         num_threads=4)

print(pred.shape)

(59599,)


In [ ]:
pd.Series(pred).sort_values(ascending = False).head(10).reset_index()['index'].to_list()

[4681, 27841, 1833, 4059, 1137, 1258, 7116, 29194, 1930, 1075]

In [ ]:
top_cols = np.argpartition(pred, -np.arange(top_N))[-top_N:][::-1]
top_cols

array([ 4681, 27841,  1833,  4059,  1137,  1258,  7116, 29194,  1930,
        1075])

In [ ]:
#рекомендации из топ 10 для нашего пользователя
recs = pd.DataFrame({'col_id': top_cols})

#теперь меняем номер строки на айди айтема и добавляем человеческое имя
recs['item_id'] = recs['col_id'].map(lightfm_mapping['items_inv_mapping'].get)
recs['title'] = recs['item_id'].map(item_titles.get)
recs

,col_id,item_id,title
0,4681,99357,"сила подсознания, или как изменить жизнь за 4 ..."
1,27841,211217,"ни сы. восточная мудрость, которая гласит: буд..."
2,1833,281005,"богатый папа, бедный папа. роберт кийосаки (об..."
3,4059,90519,думай медленно… решай быстро
4,1137,119138,одиночество в сети
5,1258,89152,50 правил умной дуры
6,7116,232537,"поступай как женщина, думай как мужчина. почем..."
7,29194,155451,выйди из зоны комфорта. измени свою жизнь
8,1930,159580,пятьдесят оттенков серого
9,1075,24551,магическая уборка. японское искусство наведени...


In [ ]:
test.head()

,user_id,item_id,progress,rating,start_date
1515866,139942,26270,58,NaN,2019-12-24
1515867,136607,82392,39,NaN,2019-12-24
1515868,142287,84446,79,NaN,2019-12-24
1515869,114933,319155,9,NaN,2019-12-24
1515870,32926,120339,73,NaN,2019-12-24


In [ ]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

In [ ]:
recs

,user_id
0,139942
1,136607
2,142287
3,114933
4,32926
...,...
1629,59961
1630,81768
1631,158991
1632,77232


In [ ]:
def generate_lightfm_recs_mapper(model, item_ids, known_items,
                                 user_features, item_features, N,
                                 user_mapping, item_inv_mapping,
                                 num_threads=4):

    def _recs_mapper(user):
        #номер строки в модели
        user_id = user_mapping[user]
        recs = model.predict(user_id, item_ids,
                             user_features=user_features,
                             item_features=item_features, num_threads=num_threads)

        additional_N = len(known_items[user_id]) if user_id in known_items else 0
        total_N = N + additional_N
        top_cols = np.argpartition(recs, -np.arange(total_N))[-total_N:][::-1]


        final_recs = [item_inv_mapping[item] for item in top_cols]
        if additional_N > 0:
            filter_items = known_items[user_id]
            final_recs = [item for item in final_recs if item not in filter_items]
        return final_recs[:N]
    return _recs_mapper

In [ ]:
known_items = train.groupby('user_id')['item_id'].apply(list).to_dict()
len(known_items)

151589

In [ ]:
known_items[100]

[41856, 131164, 214579, 66545, 145067]

In [ ]:
mapper = generate_lightfm_recs_mapper(
    lfm_model,
    item_ids=all_cols,
    known_items=known_items,
    N=top_N,
    user_features=train_user_features,
    item_features=train_items_features,
    user_mapping=lightfm_mapping['users_mapping'],
    item_inv_mapping=lightfm_mapping['items_inv_mapping'],
    num_threads=4
)

In [ ]:
%%time
recs['item_id'] = recs['user_id'].map(mapper)

CPU times: user 1min 22s, sys: 353 ms, total: 1min 23s
Wall time: 54 s


In [ ]:
recs.head()

,user_id,item_id
0,139942,"[99357, 211217, 281005, 90519, 119138, 89152, ..."
1,136607,"[99357, 58803, 281005, 32603, 90519, 308529, 1..."
2,142287,"[264997, 159580, 44081, 26963, 119138, 58803, ..."
3,114933,"[121687, 109201, 39878, 143505, 241026, 250997..."
4,32926,"[99357, 316995, 90519, 281005, 58803, 28889, 1..."


In [ ]:
recs = recs.explode('item_id')
recs['rank'] = recs.groupby('user_id').cumcount() + 1

In [ ]:
recs

,user_id,item_id,rank
0,139942,99357,1
0,139942,211217,2
0,139942,281005,3
0,139942,90519,4
0,139942,119138,5
...,...,...,...
1633,17843,4763,6
1633,17843,173707,7
1633,17843,316995,8
1633,17843,166159,9


In [ ]:
def compute_metrics(df_true, df_pred, top_N):

    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']

    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

In [ ]:
metrics = compute_metrics(test, recs, top_N)
metrics

Precision@1     0.000612
Recall@1        0.000612
Precision@2     0.000612
Recall@2        0.001224
Precision@3     0.001020
Recall@3        0.002601
Precision@4     0.000765
Recall@4        0.002601
Precision@5     0.000612
Recall@5        0.002601
Precision@6     0.000510
Recall@6        0.002601
Precision@7     0.000525
Recall@7        0.002754
Precision@8     0.000841
Recall@8        0.005508
Precision@9     0.000816
Recall@9        0.005814
Precision@10    0.000734
Recall@10       0.005814
MAP@10          0.001777
MRR             0.002068
dtype: float64

<a id="links"></a>
# Links
* https://www.benfrederickson.com/distance-metrics/ - шикарная статья про разные метрики в kNN для рекомендательных систем (Item2Item, User2User)
* https://habr.com/ru/post/486802/ - хорошая статья про разные методы (теория)
* https://github.com/dmitryhd/lightfm - форк lightfm от ребят с Авито
* https://habr.com/ru/company/avito/blog/439206/ - обзор применения указанного выше форка на recsys challenge 2018

## реранжирование

In [ ]:
#для начала моделькой генерим предсказания для train (больше часа)


%%time
recs = pd.DataFrame({
    'user_id': train['user_id'].unique()
})

mapper = generate_lightfm_recs_mapper(
    lfm_model,
    item_ids=all_cols,
    known_items=known_items,
    N=top_N,
    user_features=train_user_features,
    item_features=train_items_features,
    user_mapping=lightfm_mapping['users_mapping'],
    item_inv_mapping=lightfm_mapping['items_inv_mapping'],
    num_threads=4
)


recs['item_id'] = recs['user_id'].map(mapper)
recs = recs.explode('item_id')
recs['rank'] = recs.groupby('user_id').cumcount() + 1

metrics = compute_metrics(test, recs, top_N)
metrics

CPU times: user 1h 55min 18s, sys: 21.2 s, total: 1h 55min 39s
Wall time: 1h 11min 16s


Precision@1     0.000612
Recall@1        0.000612
Precision@2     0.000612
Recall@2        0.001224
Precision@3     0.001020
Recall@3        0.002601
Precision@4     0.000765
Recall@4        0.002601
Precision@5     0.000612
Recall@5        0.002601
Precision@6     0.000510
Recall@6        0.002601
Precision@7     0.000525
Recall@7        0.002754
Precision@8     0.000841
Recall@8        0.005508
Precision@9     0.000816
Recall@9        0.005814
Precision@10    0.000734
Recall@10       0.005814
MAP@10          0.001777
MRR             0.002068
dtype: float64

1. создать лист рекомендаций для трейна
2. добабвить фичи айтемов и юзеров
3. добавить таргет (например клик)
4. вместе с рангом засунуть в пул
5. настроить модель под ранжирование (?)
6. сделать предикт проба на таргет взаимодействия
7. отсортировать и ранжировать по пользователю по предикту

In [ ]:
from google.colab import files

# Задайте имя файла, в который вы хотите сохранить датафрейм
file_name = 'recs_train.csv'

# Сохраните датафрейм в формате CSV
recs.to_csv(file_name, index=False)

# Загрузите файл с помощью модуля files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

NameError: ignored